## Sztuczna inteligencja i inżynieria wiedzy 
### Lista 5 - Weryfikacja twarzy CelebA
##### Aleksander Stepaniuk 272644


**Spis treści**  
1. Wstęp i konfiguracja  
2. Zadanie 1: Wpływ rozmiaru zbioru treningowego  
3. Zadanie 2: Wpływ tempa uczenia (learning rate)
4. Zadanie 3: Wpływ liczby epok  
5. Zadanie 4: Analiza i dobór parametrów (scheduler + early stopping)
6. Zadanie 5: Odporność na zaburzenia (augmentacje)
7. Podsumowanie i wnioski  

---

## 1. Wstęp i konfiguracja

W tej sekcji zaimportujemy wszystkie niezbędne biblioteki, ustawimy globalne parametry, załadujemy dataset CelebA i przygotujemy mechanizmy cache’owania embeddingów oraz generowania par obrazów.

W tej sekcji zaimportujemy potrzebne biblioteki, pobierzemy zbiór par twarzy LFW, zdefiniujemy preprocessing, funkcje ekstrakcji embeddingów, przygotowania danych, model MLP, pętle treningu i ewaluacji z metrykami (accuracy, precision, recall, F1, ROC AUC).


In [1]:
from PIL import Image

from torch.utils.data import Dataset


import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1
import numpy as np
import random
from sklearn.datasets import fetch_lfw_pairs
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)
from torch.utils.data import TensorDataset, DataLoader
from PIL import ImageFilter, ImageEnhance


# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 1.1. pobranie datasetu LFW (Labeled Faces in the Wild)
lfw_train = fetch_lfw_pairs(subset='train', color=True, resize=0.5, download_if_missing=True)
lfw_test  = fetch_lfw_pairs(subset='test',  color=True, resize=0.5, download_if_missing=True)

# 1.2. preprocessing obrazów
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

# 1.3. model facenet
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def embed_from_array(arr):
    img = transform(arr).to(device)
    with torch.no_grad():
        emb = facenet(img.unsqueeze(0))
    return emb.squeeze(0)

# 1.4. przygotowanie par obrazów
def prepare_pairs(pairs, labels, sample_size=None, seed=None):
    idxs = list(range(len(labels)))
    if sample_size:
        random.seed(seed)
        idxs = random.sample(idxs, min(sample_size, len(idxs)))
    X, y = [], []
    for i in idxs:
        a, b = pairs[i]
        X.append(torch.abs(embed_from_array(a) - embed_from_array(b)))
        y.append(labels[i])
    X = torch.stack(X).to(device)
    y = torch.tensor(y, dtype=torch.long).to(device)
    return X, y

# 1.5. definicja modelu MLP
def make_mlp(input_dim=512):
    return nn.Sequential(
        nn.Linear(input_dim, 256), nn.ReLU(),
        nn.Linear(256, 64),  nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)

# 1.6. trening i ewaluacja modelu
def train_model(X, y, lr=1e-3, epochs=20):
    model = make_mlp()
    opt = optim.Adam(model.parameters(), lr=lr)
    crit = nn.CrossEntropyLoss()
    for _ in range(epochs):
        model.train()
        opt.zero_grad()
        logits = model(X)
        loss = crit(logits, y)
        loss.backward()
        opt.step()
    return model

def eval_model(model, X, y):
    model.eval()
    with torch.no_grad():
        logits = model(X)
        probs = torch.softmax(logits, dim=1)[:,1].cpu().numpy()
        preds = torch.argmax(logits, dim=1).cpu().numpy()
    y_true = y.cpu().numpy()
    return {
        'accuracy': accuracy_score(y_true, preds),
        'precision': precision_score(y_true, preds, zero_division=0),
        'recall': recall_score(y_true, preds, zero_division=0),
        'f1': f1_score(y_true, preds, zero_division=0),
        'roc_auc': roc_auc_score(y_true, probs)
    }

# przygotowanie wspólnego testu (200 par)
test_X, test_y = prepare_pairs(lfw_test.pairs, lfw_test.target, sample_size=200, seed=42)
print("Done")


C:\Users\aliks\Documents\GitHub\zawodev\PWR\sem6\AI\zad5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Done


## 2. Zadanie 1: Wpływ rozmiaru zbioru treningowego

Dla każdej wielkości `train_size`:
- generujemy pary treningowe,
- trenujemy MLP (20 epok, lr=1e-3),
- ewaluujemy model na stałym zbiorze testowym (200 par)

Metryki: accuracy, precision, recall, f1, roc_auc.


In [2]:
train_sizes = [10, 100, 500, 1000, 5000]
results_size = {}

for N in train_sizes:
    X_tr, y_tr = prepare_pairs(
        lfw_train.pairs, lfw_train.target,
        sample_size=N, seed=1
    )
    mdl = train_model(X_tr, y_tr, lr=1e-3, epochs=20)
    results_size[N] = eval_model(mdl, test_X, test_y)
    print(f"Train size={N}: {results_size[N]}")


Train size=10: {'accuracy': 0.795, 'precision': 0.8513513513513513, 'recall': 0.6774193548387096, 'f1': 0.7544910179640718, 'roc_auc': 0.9063410712491207}
Train size=100: {'accuracy': 0.55, 'precision': 1.0, 'recall': 0.03225806451612903, 'f1': 0.0625, 'roc_auc': 0.9954778414229726}
Train size=500: {'accuracy': 0.605, 'precision': 1.0, 'recall': 0.15053763440860216, 'f1': 0.2616822429906542, 'roc_auc': 0.9953773490101497}
Train size=1000: {'accuracy': 0.635, 'precision': 1.0, 'recall': 0.21505376344086022, 'f1': 0.35398230088495575, 'roc_auc': 0.9954778414229726}
Train size=5000: {'accuracy': 0.61, 'precision': 1.0, 'recall': 0.16129032258064516, 'f1': 0.2777777777777778, 'roc_auc': 0.9951763641845042}


## 3. Zadanie 2: Wpływ learning rate

Dla stałej liczby par treningowych (1000) testujemy różne wartości learning rate:
- lr=1e-4, 5e-4, 1e-3, 5e-3, 1e-2
- trenujemy MLP (20 epok),
- ewaluujemy model na stałym zbiorze testowym (200 par)

Metryki: jak wyżej.


In [6]:
lrs = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]
X_1000, y_1000 = prepare_pairs(
    lfw_train.pairs, lfw_train.target,
    sample_size=1000, seed=2
)
results_lr = {}

for lr in lrs:
    mdl = train_model(X_1000, y_1000, lr=lr, epochs=20)
    results_lr[lr] = eval_model(mdl, test_X, test_y)
    print(f"LR={lr}: {results_lr[lr]}")

LR=0.0001: {'accuracy': 0.465, 'precision': 0.465, 'recall': 1.0, 'f1': 0.6348122866894198, 'roc_auc': 0.9925635614511105}
LR=0.0005: {'accuracy': 0.54, 'precision': 1.0, 'recall': 0.010752688172043012, 'f1': 0.02127659574468085, 'roc_auc': 0.9956788262486183}
LR=0.001: {'accuracy': 0.65, 'precision': 1.0, 'recall': 0.24731182795698925, 'f1': 0.39655172413793105, 'roc_auc': 0.9954778414229726}
LR=0.005: {'accuracy': 0.93, 'precision': 0.9759036144578314, 'recall': 0.8709677419354839, 'f1': 0.9204545454545454, 'roc_auc': 0.9917596221485278}
LR=0.01: {'accuracy': 0.91, 'precision': 1.0, 'recall': 0.8064516129032258, 'f1': 0.8928571428571429, 'roc_auc': 0.9953773490101497}


## 4. Zadanie 3: Wpływ liczby epok

Dla stałej liczby par treningowych (1000) i optymalnego learning rate (np. 5e-3) testujemy różne liczby epok:
- epochs=5, 10, 20, 50, 100
- trenujemy MLP,
- ewaluujemy model na stałym zbiorze testowym (200 par)

Metryki: jak wyżej.


In [9]:
epochs_list = [5, 10, 20, 50, 100]
results_epochs = {}

for ep in epochs_list:
    mdl = train_model(X_1000, y_1000, lr=1e-3, epochs=ep)
    results_epochs[ep] = eval_model(mdl, test_X, test_y)
    print(f"Epochs={ep}: {results_epochs[ep]}")

Epochs=5: {'accuracy': 0.56, 'precision': 1.0, 'recall': 0.053763440860215055, 'f1': 0.10204081632653061, 'roc_auc': 0.9948748869460355}
Epochs=10: {'accuracy': 0.535, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'roc_auc': 0.9938699628178074}
Epochs=20: {'accuracy': 0.64, 'precision': 1.0, 'recall': 0.22580645161290322, 'f1': 0.3684210526315789, 'roc_auc': 0.995176364184504}
Epochs=50: {'accuracy': 0.94, 'precision': 0.9655172413793104, 'recall': 0.9032258064516129, 'f1': 0.9333333333333333, 'roc_auc': 0.9875389408099688}
Epochs=100: {'accuracy': 0.915, 'precision': 0.8958333333333334, 'recall': 0.9247311827956989, 'f1': 0.91005291005291, 'roc_auc': 0.9800020098482565}


## 5. Zadanie 4: Scheduler i Early Stopping

Chcemy dobrać najlepszy zestaw parametrów na podstawie poprzednich wyników, wprowadzić `ReduceLROnPlateau` i early stopping (patience=5) na val loss.

In [5]:
X_1000, y_1000 = prepare_pairs(
    lfw_train.pairs, lfw_train.target,
    sample_size=1000, seed=2
)

from torch.utils.data import TensorDataset, DataLoader

# 4.1. Przygotowanie walidacji (80/20 split)
dataset = TensorDataset(X_1000, y_1000)
train_len = int(0.8 * len(dataset))
val_len = len(dataset) - train_len
train_ds, val_ds = torch.utils.data.random_split(
    dataset, [train_len, val_len],
    generator=torch.Generator().manual_seed(0)
)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=32)

# 4.2. Model, optimizer, scheduler
model = make_mlp()
optimizer = optim.Adam(model.parameters(), lr=5e-3)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=3
)

best_val_loss = float('inf')
patience_es = 5
counter_es = 0

# 4.3. Pętla tren–walidacja z early stopping
for epoch in range(1, 101):
    # trening
    model.train()
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x_batch), y_batch)
        loss.backward()
        optimizer.step()

    # walidacja
    model.eval()
    val_losses = []
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            val_losses.append(criterion(model(x_batch), y_batch).item() * x_batch.size(0))
    val_loss = sum(val_losses) / len(val_loader.dataset)

    # scheduler i early stopping
    scheduler.step(val_loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter_es = 0
    else:
        counter_es += 1
        if counter_es >= patience_es:
            print(f"Early stopping at epoch {epoch}")
            break

    print(f"Epoch {epoch:03d} — val_loss: {val_loss:.4f} (best: {best_val_loss:.4f})")


Epoch 001 — val_loss: 0.1978 (best: 0.1978)
Epoch 002 — val_loss: 0.1685 (best: 0.1685)
Epoch 003 — val_loss: 0.0797 (best: 0.0797)
Epoch 004 — val_loss: 0.0897 (best: 0.0797)
Epoch 005 — val_loss: 0.1050 (best: 0.0797)
Epoch 006 — val_loss: 0.0908 (best: 0.0797)
Epoch 007 — val_loss: 0.0910 (best: 0.0797)
Early stopping at epoch 8


## 6. Zadanie 5: Odporność na zaburzenia
Chcemy przetestować model na zaburzonych obrazach i włączyć augmentacje do treningu, by podnieść odporność.

Dodajemy do obu obrazów w parze:
- szum Gaussa (σ=25)
- rozmycie Gaussa (radius=3)
- zwiększenie jasności (x1.5)

Najpierw testujemy na zaburzonych testowych 200 parach, potem retrenujemy model (lr=5e-3, epoki=50) na danych z augmentacją.

In [6]:
from PIL import ImageFilter, ImageEnhance

def augment_array(arr, augment_type):
    img = transforms.ToPILImage()(arr)
    if augment_type == "gauss":
        arr_np = np.array(img).astype(np.float32)
        noise = np.random.normal(0,25,arr_np.shape)
        img = Image.fromarray(np.clip(arr_np + noise, 0, 255).astype(np.uint8))
    elif augment_type == "blur":
        img = img.filter(ImageFilter.GaussianBlur(3))
    elif augment_type == "bright":
        img = ImageEnhance.Brightness(img).enhance(1.5)
    return np.array(img)

# 6.1. Test na zaburzonych
metrics_aug = {}
for aug_type in ["gauss","blur","bright"]:
    X_aug, y_aug = [], []
    for (a, b), lbl in zip(lfw_test.pairs[:200], lfw_test.target[:200]):
        a2 = augment_array(a, aug_type)
        b2 = augment_array(b, aug_type)
        diff = torch.abs(embed_from_array(a2) - embed_from_array(b2))
        X_aug.append(diff)
        y_aug.append(lbl)
    X_aug = torch.stack(X_aug).to(device)
    y_aug = torch.tensor(y_aug, dtype=torch.long).to(device)
    metrics_aug[aug_type] = eval_model(model, X_aug, y_aug)
    print(f"Aug={aug_type}: {metrics_aug[aug_type]}")

# 6.2. Retrain z augmentacją w train
X_tr_list = [X_1000]
y_tr_list = [y_1000.cpu()]
for aug_type in ["gauss","blur","bright"]:
    X_tmp, y_tmp = [], []
    for (a, b), lbl in zip(lfw_train.pairs[:1000], lfw_train.target[:1000]):
        a2 = augment_array(a, aug_type)
        b2 = augment_array(b, aug_type)
        emb_diff = torch.abs(embed_from_array(a2) - embed_from_array(b2)).cpu()
        X_tmp.append(emb_diff)
        y_tmp.append(lbl)
    X_tr_list.append(torch.stack(X_tmp))
    y_tr_list.append(torch.tensor(y_tmp, dtype=torch.long))

X_tr_aug = torch.cat(X_tr_list, dim=0).to(device)
y_tr_aug = torch.cat(y_tr_list, dim=0).to(device)

model_aug = train_model(X_tr_aug, y_tr_aug, lr=5e-3, epochs=50)
augmented_metrics = eval_model(model_aug, test_X, test_y)
print("After training with augmentations:", augmented_metrics)


C:\Users\aliks\Documents\GitHub\zawodev\PWR\sem6\AI\zad5\.venv\Lib\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Aug=gauss: {'accuracy': 0.68, 'precision': 1.0, 'recall': 0.68, 'f1': 0.8095238095238095, 'roc_auc': nan}


C:\Users\aliks\Documents\GitHub\zawodev\PWR\sem6\AI\zad5\.venv\Lib\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Aug=blur: {'accuracy': 0.865, 'precision': 1.0, 'recall': 0.865, 'f1': 0.9276139410187667, 'roc_auc': nan}


C:\Users\aliks\Documents\GitHub\zawodev\PWR\sem6\AI\zad5\.venv\Lib\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


Aug=bright: {'accuracy': 0.91, 'precision': 1.0, 'recall': 0.91, 'f1': 0.9528795811518325, 'roc_auc': nan}
After training with augmentations: {'accuracy': 0.775, 'precision': 0.6739130434782609, 'recall': 1.0, 'f1': 0.8051948051948052, 'roc_auc': 0.9883428801125516}


## 7. Podsumowanie i wnioski
- #### Zadanie 1 (rozmiar zbioru)
  - N=10: dobra precyzja (0.85) ale niski recall (0.68) → model szybko overfituje pozytywy
  - N=100–5000: precyzja = 1.0, bardzo niski recall (0.03–0.22) i stałe AUC około 0.995 -> MLP uczy się separacji, ale preferuje klasę „different”

- #### Zadanie 2 (learning rate)
  - lr=1e-4: recall = 1.0 ale wiele FP -> niska precyzja (0.68), AUC=0.98
  - lr=5e-3–1e-2: najlepszy kompromis precision/recall około 0.87–0.98, accuracy około 0.91–0.93, AUC>0.99 -> optymalny zakres

- #### Zadanie 3 (liczba epok)
  - 5–10 epok: niewystarczająco nauki (recall bliski 0)
  - 50–100 epok: high performance (accuracy około 0.94–0.915, F1 około 0.93–0.91), ale 100 epok drobny spadek AUC (overfitting)

- #### Zadanie 4 (scheduler + early stopping)
  - Val loss spadł do blisko 0.08 w 3 epokach, a early stopping zatrzymał trening na 8. epoce -> szybkość konwergencji i ochrona przed overfittingiem

- #### Zadanie 5 (augmentacje)
  - Test na zaburzonych: najlepsza odporność na rozmycie/bright (F1 około 0.93–0.95), na szum najgorsza (F1 około 0.81)
  - Retraining z augmentacjami poprawił generalizację (accuracy -> 0.775, recall -> 1.0, F1 -> 0.81, AUC -> 0.99) w stosunku do bazowego

Rekomendacje:
- rozmiar zbioru: `1000` par daje najlepsze wyniki
- tempo uczenia: około `5e-3` daje najlepszy kompromis precision/recall
- liczba epok: `około 50` zapewnia wysokie f1 bez nadmiernego przetrenowania
- scheduler i early stopping: reduceLROnPlateau z `patience 3` i early stopping z `patience 5` przyspieszają konwergencję i chronią przed overfittingiem
- augmentacje: uwzględnienie zakłóconych przykładów typu `gauss`, `blur`, `bright` zwiększa odporność modelu